# Demo Notebook how to run models on static mouse datasets

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [2]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True
dj.config['display.limit'] = 200

name = 'iclr'
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [3]:
import re
import torch
import numpy as np
import pickle 
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)
from collections import OrderedDict, Iterable
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder
from nnfabrik.utility.hypersearch import Bayesian

from nnsysident.tables.experiments import *
from nnsysident.tables.bayesian import *
from nnsysident.datasets.mouse_loaders import static_shared_loaders, static_loaders, static_loader
from nnsysident.tables.scoring import OracleScore, OracleScoreTransfer

<ipython-input-3-dce65ad1752d>:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import OrderedDict, Iterable


Connecting konstantin@134.2.168.16:3306
Schema name: konstantin_nnsysident_iclr


---

# Dataset

In [9]:
# change path here
paths = ['data/static20457-5-9-preproc0.zip']

dataset_fn = 'nnsysident.datasets.mouse_loaders.static_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=64,
    seed=1,
    file_tree=True,
)

dataloaders = builder.get_data(dataset_fn, dataset_config)

data/static20457-5-9-preproc0 exists already. Not unpacking data/static20457-5-9-preproc0.zip


In [15]:
dataloaders['train']['20457-5-9-0'].dataset.neurons.cell_motor_coordinates

array([[-301., -305.,  210.],
       [-304., -285.,  210.],
       [-283., -306.,  210.],
       ...,
       [  86., -305.,  245.],
       [ 270., -197.,  245.],
       [-222.,  -40.,  245.]])

# Model

In [5]:
model_fn = 'nnsysident.models.models.se2d_fullgaussian2d'

model_config = {'init_mu_range': 0.55,
                 'init_sigma': 0.4,
                 'input_kern': 15,
                 'hidden_kern': 13,
                 'gamma_input': 1.0,
                 'grid_mean_predictor': None,
                 'gamma_readout': 2.439}


model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)

# Trainer

In [6]:
detach_core=False  # this should be True for transfer learning

trainer_fn = 'nnsysident.training.trainers.standard_trainer'
trainer_config = dict(track_training=True, detach_core=detach_core)  
trainer = builder.get_trainer(trainer_fn, trainer_config)

# Run Training

In [7]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1)

correlation 0.0010534218
poisson_loss 4029516.2


Epoch 1: 100%|██████████| 69/69 [00:12<00:00,  5.48it/s]


[001|00/05] ---> 0.07745499163866043
correlation 0.07745499
poisson_loss 2424402.5


Epoch 2: 100%|██████████| 69/69 [00:06<00:00, 10.64it/s]


[002|00/05] ---> 0.09418176859617233
correlation 0.09418177
poisson_loss 2394455.8


...

[050|03/05] -/-> 0.20527330040931702
correlation 0.2052733
poisson_loss 2269974.5


Epoch 51: 100%|██████████| 69/69 [00:06<00:00, 10.63it/s]


[051|04/05] -/-> 0.207217738032341
correlation 0.20721774
poisson_loss 2263188.0


Epoch 52: 100%|██████████| 69/69 [00:06<00:00, 10.73it/s]


[052|05/05] -/-> 0.2079562097787857
Restoring best model after lr decay! 0.207956 ---> 0.211320
Restoring best model! 0.211320 ---> 0.211320
